### CNN on CIFR Assignment:

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use DropOut layers.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [3]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10) 

In [4]:
# Normalizing  (0 to 1)
# https://stackoverflow.com/questions/62783984/how-to-normalize-pixel-values-in-an-image-and-save-it
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255


In [5]:
def model_train(X_train, y_train, X_test, y_test, step_size, epochs):
    # Image Augmentation
    datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    iterator_train = datagen.flow(X_train, y_train, batch_size=64)

    # fit model
    steps = int(X_train.shape[0] / step_size)
    history = model.fit(iterator_train, steps_per_epoch=steps, epochs=epochs, callbacks=callbacks_list,validation_data=(X_test, y_test), verbose=1)
    
    # evaluate model
    print(model.evaluate(X_test, y_test, verbose=1))
    

In [6]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [7]:
num_filter = 12
dropout_rate = 0
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(356, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, 10, dropout_rate)
First_Transition = transition(First_Block, 128, dropout_rate)

Second_Block = denseblock(First_Transition, 10, dropout_rate)
Second_Transition = transition(Second_Block, 128, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, 64, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [8]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 356)  9612        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 356)  1424        conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 356)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [9]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [9]:
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

- Image Augmentation

In [10]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
iterator_train = datagen.flow(X_train, y_train, batch_size=64)

- Training model for 20 Epochs

In [28]:
# fit model
#steps = int(X_train.shape[0] / 64)
model.fit(iterator_train, batch_size=256, epochs=20, callbacks=callbacks_list,validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
782/782 [==============================] - 318s 384ms/step - loss: 1.4224 - accuracy: 0.4794 - val_loss: 1.8568 - val_accuracy: 0.4414

Epoch 00001: loss improved from inf to 1.42235, saving model to model.h5
Epoch 2/20
782/782 [==============================] - 287s 368ms/step - loss: 0.9651 - accuracy: 0.6592 - val_loss: 1.0305 - val_accuracy: 0.6485

Epoch 00002: loss improved from 1.42235 to 0.96511, saving model to model.h5
Epoch 3/20
782/782 [==============================] - 288s 368ms/step - loss: 0.7723 - accuracy: 0.7270 - val_loss: 0.9682 - val_accuracy: 0.6667

Epoch 00003: loss improved from 0.96511 to 0.77228, saving model to model.h5
Epoch 4/20
782/782 [==============================] - 288s 368ms/step - loss: 0.6674 - accuracy: 0.7648 - val_loss: 1.1553 - val_accuracy: 0.6281

Epoch 00004: loss improved from 0.77228 to 0.66737, saving model to model.h5
Epoch 5/20
782/782 [==============================] - 288s 368ms/step - loss: 0.5977 - accuracy: 0.7924 - va

- For 20 Epochs got Test accuracy of 85.29%.

In [29]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 21s 59ms/step - loss: 0.4543 - accuracy: 0.8529
Test loss: 0.4542732536792755
Test accuracy: 0.8529000282287598


### Loading the last Trained model and continuing Training

In [11]:
from tensorflow.keras.models import load_model

In [12]:
model = load_model('model.h5')

In [13]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

#### Training for 200 Epochs (But early stopped at 102 Epoch)

In [15]:
model.fit(iterator_train, batch_size=256, epochs=200, callbacks=callbacks_list,validation_data=(X_test, y_test), verbose=1)

Epoch 1/200
782/782 [==============================] - 319s 385ms/step - loss: 0.1369 - accuracy: 0.9514 - val_loss: 0.5026 - val_accuracy: 0.8670

Epoch 00001: loss improved from inf to 0.13691, saving model to model.h5
Epoch 2/200
782/782 [==============================] - 289s 369ms/step - loss: 0.1384 - accuracy: 0.9500 - val_loss: 0.4711 - val_accuracy: 0.8732

Epoch 00002: loss did not improve from 0.13691
Epoch 3/200
782/782 [==============================] - 282s 361ms/step - loss: 0.1328 - accuracy: 0.9528 - val_loss: 0.4438 - val_accuracy: 0.8752

Epoch 00003: loss improved from 0.13691 to 0.13276, saving model to model.h5
Epoch 4/200
782/782 [==============================] - 267s 342ms/step - loss: 0.1277 - accuracy: 0.9543 - val_loss: 0.4569 - val_accuracy: 0.8773

Epoch 00004: loss improved from 0.13276 to 0.12766, saving model to model.h5
Epoch 5/200
782/782 [==============================] - 271s 347ms/step - loss: 0.1250 - accuracy: 0.9546 - val_loss: 0.7220 - val_accu

#### Continuing Training for 100 more Epochs (Total Epochs 20+102+100 = 222 Epochs)

In [14]:
model.fit(iterator_train, batch_size=256, epochs=100, callbacks=callbacks_list,validation_data=(X_test, y_test), verbose=1)

Epoch 1/100
782/782 [==============================] - 315s 381ms/step - loss: 0.0493 - accuracy: 0.9825 - val_loss: 0.4888 - val_accuracy: 0.8949

Epoch 00001: loss improved from inf to 0.04929, saving model to model.h5
Epoch 2/100
782/782 [==============================] - 285s 364ms/step - loss: 0.0468 - accuracy: 0.9833 - val_loss: 0.4963 - val_accuracy: 0.8966

Epoch 00002: loss improved from 0.04929 to 0.04675, saving model to model.h5
Epoch 3/100
782/782 [==============================] - 285s 364ms/step - loss: 0.0526 - accuracy: 0.9821 - val_loss: 0.5100 - val_accuracy: 0.8910

Epoch 00003: loss did not improve from 0.04675
Epoch 4/100
782/782 [==============================] - 285s 364ms/step - loss: 0.0442 - accuracy: 0.9847 - val_loss: 0.5261 - val_accuracy: 0.8936

Epoch 00004: loss improved from 0.04675 to 0.04422, saving model to model.h5
Epoch 5/100
782/782 [==============================] - 285s 364ms/step - loss: 0.0467 - accuracy: 0.9827 - val_loss: 0.5353 - val_accu

In [15]:
# Test the model
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 21s 58ms/step - loss: 0.5514 - accuracy: 0.9003
Test loss: 0.551444947719574
Test accuracy: 0.9003000259399414


# After Training model (222 Epochs) got the Test Accuracy of __90.03%.__